In [2]:
!git clone https://github.com/Blackzxy/text_sum_sim.git

fatal: destination path 'text_sum_sim' already exists and is not an empty directory.


In [ ]:
import pandas as pd

In [ ]:
wiki_df = pd.read_csv('/content/drive/MyDrive/wiki_dataset.csv')

In [ ]:
len(wiki_df)

56312

In [ ]:
wiki_df.head()

In [ ]:
wiki_df.iloc[0]['simple']

In [ ]:
wiki_df.iloc[0]['hard']

In [ ]:
wiki_parag_aligned_df = pd.read_csv('/content/drive/MyDrive/wiki_paragraph_aligned.csv')

In [ ]:
wiki_parag_aligned_df.head()

In [ ]:
len(wiki_parag_aligned_df)

67065

In [ ]:
import torch

In [ ]:
df_size = len(wiki_parag_aligned_df)
train_size = int(0.8*df_size)
valid_size =  int(0.1*df_size)
test_size = df_size - train_size - valid_size

In [ ]:
train_size, valid_size, test_size

(53652, 6706, 6707)

In [ ]:
import numpy as np

train_paragh, test_paragh, valid_paragh = np.split(
    wiki_parag_aligned_df.sample(frac = 1, random_state=42),
    [int(.8*len(wiki_parag_aligned_df)), int(.9*len(wiki_parag_aligned_df))]
)

In [ ]:
len(train_paragh), len(test_paragh), len(valid_paragh)

(53652, 6706, 6707)

In [ ]:
train_paragh.reset_index().head()

In [ ]:
complex_list = []
simple_list = []


for idx, row in train_paragh.reset_index().iterrows():
  #print(train_paragh.iloc[idx]['hard'])
  complex_list.append(train_paragh.iloc[idx]['hard'])
  simple_list.append(train_paragh.iloc[idx]['simple'])

file_write_obj = open('/content/wiki_paragh.train.complex','w', encoding = 'utf-8')
for var in complex_list:
  file_write_obj.write(var)
  file_write_obj.write('\n')
file_write_obj.close()

In [ ]:


file_write_obj = open('/content/wiki_paragh.train.simple','w', encoding = 'utf-8')
for var in simple_list:
  file_write_obj.write(var)
  file_write_obj.write('\n')
file_write_obj.close()


In [ ]:
complex_list = []
simple_list = []


for idx, row in test_paragh.reset_index().iterrows():
  #print(train_paragh.iloc[idx]['hard'])
  complex_list.append(test_paragh.iloc[idx]['hard'])
  simple_list.append(test_paragh.iloc[idx]['simple'])

file_write_obj = open('/content/wiki_paragh.test.complex','w', encoding = 'utf-8')
for var in complex_list:
  file_write_obj.write(var)
  file_write_obj.write('\n')
file_write_obj.close()

file_write_obj = open('/content/wiki_paragh.test.simple','w', encoding = 'utf-8')
for var in simple_list:
  file_write_obj.write(var)
  file_write_obj.write('\n')
file_write_obj.close()


In [ ]:
complex_list = []
simple_list = []


for idx, row in valid_paragh.reset_index().iterrows():
  #print(train_paragh.iloc[idx]['hard'])
  complex_list.append(valid_paragh.iloc[idx]['hard'])
  simple_list.append(valid_paragh.iloc[idx]['simple'])

file_write_obj = open('/content/wiki_paragh.valid.complex','w', encoding = 'utf-8')
for var in complex_list:
  file_write_obj.write(var)
  file_write_obj.write('\n')
file_write_obj.close()

file_write_obj = open('/content/wiki_paragh.valid.simple','w', encoding = 'utf-8')
for var in simple_list:
  file_write_obj.write(var)
  file_write_obj.write('\n')
file_write_obj.close()


In [ ]:
!nvidia-smi

In [3]:
!git clone https://github.com/feralvam/easse.git
%cd easse
%pip install -e .

C:\Users\z1325\Desktop\NLP_Torch\text-summarization\GitHub_T5\easse


fatal: destination path 'easse' already exists and is not an empty directory.


Obtaining file:///C:/Users/z1325/Desktop/NLP_Torch/text-summarization/GitHub_T5/easse
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Cloning https://github.com/facebookresearch/text-simplification-evaluation.git to c:\users\z1325\appdata\local\temp\pip-install-7fb5fcp0\tseval_ca385e6db22649b7acfce865ee1513d0
  Resolved https://github.com/facebookresearch/text-simplification-evaluation.git to commit f335e2e27026321c7c3d1dd63857416c7e7397b2
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Cloning https://github.com/cisnlp/simalign.git to c:\users\z1325\appdata\local\temp\pip-install-7fb5fcp0\simalign_2c605fbe04744224a36ce78b63bff76a
  Resolved https://github.com/cisnlp/simalign.git to commit 05332bf2f6ccde075c3aba94248d6105d9f95a00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started

  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/text-simplification-evaluation.git 'C:\Users\z1325\AppData\Local\Temp\pip-install-7fb5fcp0\tseval_ca385e6db22649b7acfce865ee1513d0'
  Running command git clone --filter=blob:none --quiet https://github.com/cisnlp/simalign.git 'C:\Users\z1325\AppData\Local\Temp\pip-install-7fb5fcp0\simalign_2c605fbe04744224a36ce78b63bff76a'



  Attempting uninstall: easse
    Found existing installation: easse 0.2.4
    Uninstalling easse-0.2.4:
      Successfully uninstalled easse-0.2.4
  Running setup.py develop for easse


In [ ]:
%pip install -e .

In [ ]:
from easse.sari import corpus_sari

In [ ]:
# -- fix path --
from pathlib import Path
import sys
# -- end fix path --
from functools import lru_cache
from multiprocessing import Pool, Lock
from string import punctuation
import multiprocessing
import Levenshtein
import numpy as np
import spacy
import nltk
import shutil
import time
import pickle
import hashlib

nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from sacremoses import MosesDetokenizer, MosesTokenizer


REPO_DIR = Path('/content/text_sum_sim')
RESOURCES_DIR = REPO_DIR / 'resources'
EXP_DIR = REPO_DIR / 'experiments'
DATASETS_DIR = RESOURCES_DIR / 'datasets'
PROCESSED_DATA_DIR = RESOURCES_DIR / "processed_data"
DUMPS_DIR = RESOURCES_DIR / "DUMPS"
OUTPUT_DIR = REPO_DIR / 'output'


ASSET_DATASET = "asset"
WIKILARGE_DATASET = 'wikilarge'
WIKI_DATASET = 'wiki'
TURKCORPUS_DATASET = "turkcorpus"
NEWSELA_DATASET = "newsela"
MODEL_NAME = 't5-base'

WORD_EMBEDDINGS_NAME = "glove.42B.300d"
WORD_FREQUENCY_FILEPATH = RESOURCES_DIR /'enwiki-words-frequency.txt'


LANGUAGES = ['complex', 'simple']
PHASES = ['train_small', 'tuning_small', 'test_small']

# from source.helper import tokenize, yield_lines, load_dump, dump, write_lines, count_line, \
#     print_execution_time, save_preprocessor, yield_sentence_pair

stopwords = set(stopwords.words('english'))

#######################
def get_tokenizer():
    return MosesTokenizer(lang='en')

def get_detokenizer():
    return MosesDetokenizer(lang='en')

def tokenize(sentence):
    return get_tokenizer().tokenize(sentence)

def write_lines(lines, filepath):
    filepath = Path(filepath)
    filepath.parent.mkdir(parents=True, exist_ok=True)
    with filepath.open("w") as fout:
        for line in lines:
            fout.write(line + '\n')


def read_lines(filepath):
    return [line.rstrip() for line in yield_lines(filepath)]


def yield_lines(filepath):
    filepath = Path(filepath)
    with filepath.open('r') as f:
        for line in f:
            yield line.rstrip()


def yield_sentence_pair_with_index(filepath1, filepath2):
    index = 0
    with Path(filepath1).open('r') as f1, Path(filepath2).open('r') as f2:
        for line1, line2 in zip(f1, f2):
            index += 1
            yield (line1.rstrip(), line2.rstrip(), index)
            

def yield_sentence_pair(filepath1, filepath2):
    with Path(filepath1).open('r') as f1, Path(filepath2).open('r') as f2:
        for line1, line2 in zip(f1, f2):
            yield line1.rstrip(), line2.rstrip()


def count_line(filepath):
    filepath = Path(filepath)
    line_count = 0
    with filepath.open("r") as f:
        for line in f:
            line_count += 1
    return line_count


def load_dump(filepath):
    return pickle.load(open(filepath, 'rb'))


def dump(obj, filepath):
    pickle.dump(obj, open(filepath, 'wb'))

def save_preprocessor(preprocessor):
    DUMPS_DIR.mkdir(parents=True, exist_ok=True)
    PREPROCESSOR_DUMP_FILE = DUMPS_DIR / 'preprocessor.pickle'
    dump(preprocessor, PREPROCESSOR_DUMP_FILE)


def load_preprocessor():
    PREPROCESSOR_DUMP_FILE = DUMPS_DIR / 'preprocessor.pickle'
    if PREPROCESSOR_DUMP_FILE.exists():
        return load_dump(PREPROCESSOR_DUMP_FILE)
    else:
        return None

def generate_hash(data):
    h = hashlib.new('md5')
    h.update(str(data).encode())
    return h.hexdigest()

def get_data_filepath(dataset, phase, type, i=None):
    suffix = ''
    if i is not None:
        suffix = f'.{i}'
    filename = f'{dataset}.{phase}.{type}{suffix}'
    return DATASETS_DIR / dataset / filename

#################################

def round(val):
    return '%.2f' % val


def safe_division(a, b):
    return a / b if b else 0


# def tokenize(sentence):
#     return sentence.split()

def is_punctuation(word):
    return ''.join([char for char in word if char not in punctuation]) == ''


def remove_punctuation(text):
    return ' '.join([word for word in tokenize(text) if not is_punctuation(word)])


def remove_stopwords(text):
    return ' '.join([w for w in tokenize(text) if w.lower() not in stopwords])


def get_dependency_tree_depth(sentence):
    def tree_height(node):
        if len(list(node.children)) == 0:
            return 0
        return 1 + max(tree_height(child) for child in node.children)

    tree_depths = [tree_height(spacy_sentence.root) for spacy_sentence in spacy_process(sentence).sents]
    if len(tree_depths) == 0:
        return 0
    return max(tree_depths)

def get_spacy_model():
    model = 'en_core_web_sm'
    if not spacy.util.is_package(model):
        spacy.cli.download(model)
        spacy.cli.link(model, model, force=True, model_path=spacy.util.get_package_path(model))
    return spacy.load(model)


def spacy_process(text):
    return get_spacy_model()(str(text))


def get_word2rank(vocab_size=np.inf):
    model_filepath = DUMPS_DIR / f"{WORD_EMBEDDINGS_NAME}.pk"
    if model_filepath.exists():
        return load_dump(model_filepath)


def get_normalized_rank(word):
    max = len(get_word2rank())
    rank = get_word2rank().get(word, max)
    return np.log(1 + rank) / np.log(1 + max)

    

def get_complexity_score2(sentence):
    words = tokenize(remove_stopwords(remove_punctuation(sentence)))
    words = [word for word in words if word in get_word2rank()]  # remove unknown words
    if len(words) == 0:
        return 1.0
    return np.array([get_normalized_rank(word) for word in words]).mean()

def get_word_frequency():
    model_filepath = DUMPS_DIR / f'{WORD_FREQUENCY_FILEPATH.stem}.pk'
    if model_filepath.exists():
        return load_dump(model_filepath)
    else:
        DUMPS_DIR.mkdir(parents=True, exist_ok=True) 
        word_freq = {}
        for line in yield_lines(WORD_FREQUENCY_FILEPATH):
            chunks = line.split(' ')
            word = chunks[0]
            freq = int(chunks[1])
            word_freq[word] = freq
        dump(word_freq, model_filepath)
        return word_freq


def get_normalized_frequency(word):
    max = 153141437 # the 153141437, the max frequency
    freq = get_word_frequency().get(word, 0)
    return 1.0 - np.log(1 + freq) / np.log(1 + max)


def get_complexity_score(sentence):
    # words = tokenize(remove_stopwords(remove_punctuation(sentence)))
    words = tokenize(remove_punctuation(sentence))
    words = [word for word in words if word in get_word2rank()]  # remove unknown words
    if len(words) == 0:
        return 1.0
    
    return np.array([get_normalized_frequency(word.lower()) for word in words]).mean()




class RatioFeature:
    def __init__(self, feature_extractor, target_ratio=0.8):
        self.feature_extractor = feature_extractor
        self.target_ratio = target_ratio

    def encode_sentence(self, sentence):
        return f'{self.name}_{self.target_ratio}'

    def encode_sentence_pair(self, complex_sentence, simple_sentence):
        return f'{self.name}_{self.feature_extractor(complex_sentence, simple_sentence)}', simple_sentence

    def decode_sentence(self, encoded_sentence):
        return encoded_sentence

    @property
    def name(self):
        class_name = self.__class__.__name__.replace('RatioFeature', '')
        name = ""
        for word in re.findall('[A-Z][^A-Z]*', class_name):
            if word: name += word[0]
        if not name: name = class_name
        return name


class WordRatioFeature(RatioFeature):
    def __init__(self, *args, **kwargs):
        super().__init__(self.get_word_length_ratio, *args, **kwargs)

    def get_word_length_ratio(self, complex_sentence, simple_sentence):
        return round(safe_division(len(tokenize(simple_sentence)), len(tokenize(complex_sentence))))


class CharRatioFeature(RatioFeature):
    def __init__(self, *args, **kwargs):
        super().__init__(self.get_char_length_ratio, *args, **kwargs)

    def get_char_length_ratio(self, complex_sentence, simple_sentence):
        return round(safe_division(len(simple_sentence), len(complex_sentence)))


class LevenshteinRatioFeature(RatioFeature):
    def __init__(self, *args, **kwargs):
        super().__init__(self.get_levenshtein_ratio, *args, **kwargs)

    def get_levenshtein_ratio(self, complex_sentence, simple_sentence):
        return round(Levenshtein.ratio(complex_sentence, simple_sentence))


class WordRankRatioFeature(RatioFeature):
    def __init__(self, *args, **kwargs):
        super().__init__(self.get_word_rank_ratio, *args, **kwargs)

    def get_word_rank_ratio(self, complex_sentence, simple_sentence):
        return round(min(safe_division(self.get_lexical_complexity_score(simple_sentence),
                                       self.get_lexical_complexity_score(complex_sentence)), 2))

    def get_lexical_complexity_score(self, sentence):
        words = tokenize(remove_stopwords(remove_punctuation(sentence)))
        words = [word for word in words if word in get_word2rank()]
        if len(words) == 0:
            return np.log(1 + len(get_word2rank()))
        return np.quantile([self.get_rank(word) for word in words], 0.75)

    
    def get_rank(self, word):
        rank = get_word2rank().get(word, len(get_word2rank()))
        return np.log(1 + rank)


class DependencyTreeDepthRatioFeature(RatioFeature):
    def __init__(self, *args, **kwargs):
        super().__init__(self.get_dependency_tree_depth_ratio, *args, **kwargs)

    def get_dependency_tree_depth_ratio(self, complex_sentence, simple_sentence):
        return round(
            safe_division(self.get_dependency_tree_depth(simple_sentence),
                          self.get_dependency_tree_depth(complex_sentence)))

    
    def get_dependency_tree_depth(self, sentence):
        def get_subtree_depth(node):
            if len(list(node.children)) == 0:
                return 0
            return 1 + max([get_subtree_depth(child) for child in node.children])

        tree_depths = [get_subtree_depth(spacy_sentence.root) for spacy_sentence in self.spacy_process(sentence).sents]
        if len(tree_depths) == 0:
            return 0
        return max(tree_depths)

    def spacy_process(self, text):
        return get_spacy_model()(text)


class Preprocessor:
    def __init__(self, features_kwargs=None):
        super().__init__()

        self.features = self.get_features(features_kwargs)
        if features_kwargs:
            self.hash = generate_hash(str(features_kwargs).encode())
        else:
            self.hash = "no_feature"

    def get_class(self, class_name, *args, **kwargs):
        return globals()[class_name](*args, **kwargs)

    def get_features(self, feature_kwargs):
        features = []
        for feature_name, kwargs in feature_kwargs.items():
            features.append(self.get_class(feature_name, **kwargs))
        return features

    def encode_sentence(self, sentence):
        if self.features:
            line = ''
            for feature in self.features:
                line += feature.encode_sentence(sentence) + ' '
            line += ' ' + sentence
            return line.rstrip()
        else:
            return sentence

    def encode_sentence_pair(self, complex_sentence, simple_sentence):
        # print(complex_sentence)
        if self.features:
            line = ''
            for feature in self.features:
                processed_complex, _ = feature.encode_sentence_pair(complex_sentence, simple_sentence)
                line += processed_complex + ' '
            line += ' ' + complex_sentence
            return line.rstrip()

        else:
            return complex_sentence

    def decode_sentence(self, encoded_sentence):
        for feature in self.features:
            decoded_sentence = feature.decode_sentence(encoded_sentence)
        return decoded_sentence

    # def encode_file(self, input_filepath, output_filepath):
    #     with open(output_filepath, 'w') as f:
    #         for line in yield_lines(input_filepath):
    #             f.write(self.encode_sentence(line) + '\n')

    # def decode_file(self, input_filepath, output_filepath):
    #     with open(output_filepath, 'w') as f:
    #         for line in yield_lines(input_filepath):
    #             f.write(self.decode_sentence(line) + '\n')

    def process_encode_sentence_pair(self, sentences):
        print(f"{sentences[2]}/{self.line_count}", sentences[0])  # sentence[0] index
        return (self.encode_sentence_pair(sentences[0], sentences[1]))

    def pool_encode_sentence_pair(self, args):
        # print(f"{processed_line_count}/{self.line_count}")
        complex_sent, simple_sent, queue = args
        queue.put(1)
        return self.encode_sentence_pair(complex_sent, simple_sent)
    
    def encode_file_pair(self, complex_filepath, simple_filepath):
        # print(f"Preprocessing file: {complex_filepath}")
        processed_complex_sentences = []
        self.line_count = count_line(simple_filepath)

        nb_cores = multiprocessing.cpu_count()
        manager = multiprocessing.Manager()
        queue = manager.Queue()

        pool = Pool(processes=nb_cores)
        args = [(complex_sent, simple_sent, queue) for complex_sent, simple_sent in
                yield_sentence_pair(complex_filepath, simple_filepath)]
        res = pool.map_async(self.pool_encode_sentence_pair, args)
        while not res.ready():
            # remaining = res._number_left * res._chunksize
            size = queue.qsize()
            print(f"Preprocessing: {size} / {self.line_count}")
            time.sleep(0.5)
        encoded_sentences = res.get()
        pool.close()
        pool.join()

        return encoded_sentences

    def get_preprocessed_filepath(self, dataset, phase, type):
        filename = f'{dataset}.{phase}.{type}'
        return self.preprocessed_data_dir / filename

    def preprocess_dataset(self, dataset):
        # download_requirements()
        self.preprocessed_data_dir = PROCESSED_DATA_DIR / self.hash / dataset
        self.preprocessed_data_dir.mkdir(parents=True, exist_ok=True)
        save_preprocessor(self)
        print(f'Preprocessing dataset: {dataset}')

        for phase in PHASES:
            # for phase in ["valid", "test"]:
            complex_filepath = get_data_filepath(dataset, phase, 'complex')
            simple_filepath = get_data_filepath(dataset, phase, 'simple')

            complex_output_filepath = self.preprocessed_data_dir / complex_filepath.name
            simple_output_filepath = self.preprocessed_data_dir / simple_filepath.name
            if complex_output_filepath.exists() and simple_output_filepath.exists():
                continue

            print(f'Prepocessing files: {complex_filepath.name} {simple_filepath.name}')
            processed_complex_sentences = self.encode_file_pair(complex_filepath, simple_filepath)

            write_lines(processed_complex_sentences, complex_output_filepath)
            shutil.copy(simple_filepath, simple_output_filepath)

        print(f'Preprocessing dataset "{dataset}" is finished.')
        return self.preprocessed_data_dir



# if __name__ == '__main__':
#     features_kwargs = {
#         # 'WordRatioFeature': {'target_ratio': 0.8},
#         'CharRatioFeature': {'target_ratio': 0.8},
#         'LevenshteinRatioFeature': {'target_ratio': 0.8},
#         'WordRankRatioFeature': {'target_ratio': 0.8},
#         'DependencyTreeDepthRatioFeature': {'target_ratio': 0.8}
#     }
#     # features_kwargs = {}
#     preprocessor = Preprocessor(features_kwargs)
#     # preprocessor.preprocess_dataset(WIKILARGE_DATASET)
#     preprocessor.preprocess_dataset(TURKCORPUS_DATASET)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
%pip install numpy
%pip install pytorch_lightning
%pip install transformers


In [ ]:
%pip install optuna

In [ ]:

from functools import lru_cache
from pathlib import Path
from os.path import join, isfile
import optuna
from optuna.integration import PyTorchLightningPruningCallback
from easse.sari import corpus_sari
from torch.nn import functional as F
# from preprocessor import tokenize, yield_sentence_pair, yield_lines, load_preprocessor, read_lines, \
#     count_line
import argparse
from argparse import ArgumentParser
import os
import logging
import random
import nltk
# from preprocessor import  get_data_filepath, TURKCORPUS_DATASET, NEWSELA_DATASET

nltk.download('punkt')

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.trainer import seed_everything
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast,
    get_linear_schedule_with_warmup, AutoConfig, AutoModel
)

class T5FineTuner(pl.LightningModule):
    def __init__(self,args):
    # def __init__(self, model_name, learning_rate, adam_epsilon, custom_loss, weight_decay, dataset,
    #              train_batch_size, valid_batch_size, train_sample_size, valid_sample_size, max_seq_length,
    #              n_gpu, gradient_accumulation_steps, num_train_epochs, warmup_steps, nb_sanity_val_steps,
    #              *args, **kwargs):
        super(T5FineTuner, self).__init__()
        self.args = args
        self.save_hyperparameters()
        self.model = T5ForConditionalGeneration.from_pretrained(self.args.model_name)
        self.tokenizer = T5TokenizerFast.from_pretrained(self.args.model_name)
        self.model = self.model.to(self.device)
        self.preprocessor = load_preprocessor()


    def is_logger(self):
        return self.trainer.global_rank <= 0

    def forward(
        self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
    ):
        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )
        return outputs

    def generate(self, sentence):
        sentence = self.preprocessor.encode_sentence(sentence)
        text = "simplify: " + sentence

        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.args.max_seq_length,
            padding='max_length',
            return_tensors="pt"
        )

        input_ids = encoding["input_ids"].to(self.device)
        attention_masks = encoding["attention_mask"].to(self.device)

        beam_outputs = self.model.generate(
            input_ids=input_ids,
            attention_mask=attention_masks,
            do_sample=False,
            max_length=self.args.max_seq_length,
            num_beams=8,
            top_k=120,
            top_p=0.98,
            early_stopping=True,
            num_return_sequences=1
        )
        pred_sent = self.tokenizer.decode(beam_outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        return pred_sent

    def training_step(self, batch, batch_idx):
        labels = batch["target_ids"]
        # Huggingface’s loss functions are defined to exclude the ID -100 during loss calculations. Therefore, we need to convert all padding token IDs in labels to -100.
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        self.opt.zero_grad()
        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            labels=labels,
            decoder_attention_mask=batch['target_mask'],
        )

        if self.args.custom_loss:
            loss = outputs.loss
            complexity_score = random.randint(0, 100) * 0.01
            # complexity_score = self._custom_step(outputs['logits'])
            # loss = loss * complexity_score
            lambda_ = 0.7
            # loss = lambda_ * loss + (1-lambda_)*complexity_score
            # loss = torch.sqrt(loss + lambda_ * complexity_score)
            loss = loss + complexity_score + lambda_ * (complexity_score - loss)
            
            print(complexity_score)
            self.log('train_loss', loss, on_step=True, prog_bar=True, logger=True)
            # print(loss)
            return loss
        else:
            loss = outputs.loss
            self.log('train_loss', loss, on_step=True, prog_bar=True, logger=True)
            return loss


    def validation_step(self, batch, batch_idx):
        loss = self.sari_validation_step(batch)
        # loss = self._step(batch)
        print("Val_loss", loss)
        logs = {"val_loss": loss}

        self.log('val_loss', loss)
        return torch.tensor(loss, dtype=float)

    def sari_validation_step(self, batch):
        def generate(sentence):
            sentence = self.preprocessor.encode_sentence(sentence)
            text = "simplify: " + sentence

            encoding = self.tokenizer(
                text,
                truncation=True,
                max_length=self.args.max_seq_length,
                padding='max_length',
                return_tensors="pt"
            )

            input_ids = encoding["input_ids"].to(self.device)
            attention_masks = encoding["attention_mask"].to(self.device)

            # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
            beam_outputs = self.model.generate(
                input_ids=input_ids,
                attention_mask=attention_masks,
                do_sample=False,
                max_length=self.args.max_seq_length,
                num_beams=8,
                top_k=120,
                top_p=0.98,
                early_stopping=True,
                num_return_sequences=1
            ).to(self.device)
            # final_outputs = []
            # for beam_output in beam_outputs:
            sent = self.tokenizer.decode(beam_outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
            # if sent.lower() != sentence.lower() and sent not in final_outputs:
                # final_outputs.append(sent)
            
            return sent
            # return final_outputs[0]

        pred_sents = []
        for source in batch["source"]:
            pred_sent = generate(source)
            pred_sents.append(pred_sent)

        score = corpus_sari(batch["source"], pred_sents, [batch["targets"]])
        print("Sari score: ", score)

        return 1 - score / 100

    def configure_optimizers(self):
        "Prepare optimizer and schedule (linear warmup and decay)"

        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.args.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.args.learning_rate, eps=self.args.adam_epsilon)
        # optimizer = SAM(optimizer_grouped_parameters, base_optimizer, lr=self.hparams.learning_rate, momentum=0.9)
        self.opt = optimizer
        return [optimizer]

    def optimizer_step(self, epoch=None, batch_idx=None, optimizer=None, optimizer_idx=None, optimizer_closure=None,
                       on_tpu=None, using_native_amp=None, using_lbfgs=None):
        optimizer.step(closure=optimizer_closure)

        optimizer.zero_grad()
        self.lr_scheduler.step()
    
    def save_core_model(self):
      tmp = self.args.model_name + 'core'
      store_path = OUTPUT_DIR / tmp
      self.model.save_pretrained(store_path)
      self.tokenizer.save_pretrained(store_path)



    def train_dataloader(self):
        train_dataset = TrainDataset(dataset=self.args.dataset,
                                     tokenizer=self.tokenizer,
                                     max_len=self.args.max_seq_length,
                                     sample_size=self.args.train_sample_size)

        dataloader = DataLoader(train_dataset,
                                batch_size=self.args.train_batch_size,
                                drop_last=True,
                                shuffle=True,
                                pin_memory=True,
                                num_workers=4)
        t_total = ((len(dataloader.dataset) // (self.args.train_batch_size * max(1, self.args.n_gpu)))
                   // self.args.gradient_accumulation_steps
                   * float(self.args.num_train_epochs)
                   )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.args.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler
        return dataloader

    def val_dataloader(self):
        val_dataset = ValDataset(dataset=self.args.dataset,
                                 tokenizer=self.tokenizer,
                                 max_len=self.args.max_seq_length,
                                 sample_size=self.args.valid_sample_size)
        return DataLoader(val_dataset,
                          batch_size=self.args.valid_batch_size,
                          num_workers=4)
    @staticmethod
    def add_model_specific_args(parent_parser):
      p = ArgumentParser(parents=[parent_parser],add_help = False)
      p.add_argument('-m','--model_name', default='t5-base')
      p.add_argument('-TrainBS','--train_batch_size',type=int, default=6)
      p.add_argument('-ValidBS','--valid_batch_size',type=int, default=6)
      p.add_argument('-lr','--learning_rate',type=float, default=3e-4)
      p.add_argument('-MaxSeqLen','--max_seq_length',type=int, default=256)
      p.add_argument('-AdamEps','--adam_epsilon', default=1e-8)
      p.add_argument('-WeightDecay','--weight_decay', default = 0.1)
      p.add_argument('-WarmupSteps','--warmup_steps',default=5)
      p.add_argument('-NumEpoch','--num_train_epochs',default=2)
      p.add_argument('-CosLoss','--custom_loss', default=False)
      p.add_argument('-GradAccuSteps','--gradient_accumulation_steps', default=1)
      p.add_argument('-GPUs','--n_gpu',default=torch.cuda.device_count())
      p.add_argument('-nbSVS','--nb_sanity_val_steps',default = -1)
      p.add_argument('-TrainSampleSize','--train_sample_size', default=1)
      p.add_argument('-ValidSampleSize','--valid_sample_size', default=1)
      return p


logger = logging.getLogger(__name__)


class LoggingCallback(pl.Callback):
    def on_validation_end(self, trainer, pl_module):
        logger.info("***** Validation results *****")
        if pl_module.is_logger():
            metrics = trainer.callback_metrics
            # Log results
            for key in sorted(metrics):
                if key not in ["log", "progress_bar"]:
                    logger.info("{} = {}\n".format(key, str(metrics[key])))

    def on_test_end(self, trainer, pl_module):
        logger.info("***** Test results *****")

        if pl_module.is_logger():
            metrics = trainer.callback_metrics

            # Log and save results to file
            output_test_results_file = os.path.join(pl_module.args.output_dir, "test_results.txt")
            with open(output_test_results_file, "w") as writer:
                for key in sorted(metrics):
                    if key not in ["log", "progress_bar"]:
                        logger.info("{} = {}\n".format(key, str(metrics[key])))
                        writer.write("{} = {}\n".format(key, str(metrics[key])))


class TrainDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_len=256, sample_size=1):
        self.sample_size = sample_size
        # print("init TrainDataset ...")
        preprocessor = load_preprocessor()
        self.source_filepath = preprocessor.get_preprocessed_filepath(dataset, 'train_small', 'complex')
        self.target_filepath = preprocessor.get_preprocessed_filepath(dataset, 'train_small', 'simple')

        self.max_len = max_len
        self.tokenizer = tokenizer

        self._load_data()

    def _load_data(self):
        self.inputs = read_lines(self.source_filepath)
        self.targets = read_lines(self.target_filepath)

    def __len__(self):
        return int(len(self.inputs) * self.sample_size)

    def __getitem__(self, index):
        source = "simplify: " + self.inputs[index]
        target = self.targets[index]

        tokenized_inputs = self.tokenizer(
            [source],
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors="pt"
        )
        tokenized_targets = self.tokenizer(
            [target],
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors="pt"
        )
        source_ids = tokenized_inputs["input_ids"].squeeze()
        target_ids = tokenized_targets["input_ids"].squeeze()

        src_mask = tokenized_inputs["attention_mask"].squeeze()  # might need to squeeze
        target_mask = tokenized_targets["attention_mask"].squeeze()  # might need to squeeze

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask,
                'sources': self.inputs[index], 'targets': [self.targets[index]]}


class ValDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_len=256, sample_size=1):
        self.sample_size = sample_size
        self.source_filepath = get_data_filepath(dataset, 'tuning_small', 'complex')
        self.target_filepaths = get_data_filepath(dataset, 'tuning_small', 'simple')
        # self.source_filepath = get_data_filepath(dataset, 'valid', 'complex')
        # self.target_filepaths = get_data_filepath(dataset, 'valid', 'simple')
        # if dataset == NEWSELA_DATASET:
        #     self.target_filepaths = [get_data_filepath(dataset, 'valid', 'simple')]

        # else:  # TURKCORPUS_DATASET as default
        #     self.target_filepaths = [get_data_filepath(TURKCORPUS_DATASET, 'valid', 'simple.turk', i) for i in range(8)]

        self.max_len = max_len
        self.tokenizer = tokenizer

        self._build()

    def __len__(self):
        return int(len(self.inputs) * self.sample_size)

    def __getitem__(self, index):
        return {"source": self.inputs[index], "targets": self.targets[index]}

    def _build(self):
        self.inputs = []
        self.targets = []

        for source in yield_lines(self.source_filepath):
            self.inputs.append(source)
        
        for target in yield_lines(self.target_filepaths):
            self.targets.append(target)


        # self.targets = [[] for _ in range(count_line(self.target_filepaths[0]))]
        # for filepath in self.target_filepaths:
        #     for idx, line in enumerate(yield_lines(filepath)):
        #         self.targets[idx].append(line)


def train(args):
    #args = argparse.Namespace(**train_args)
    seed_everything(args.seed)

    #print(train_args)
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        dirpath=args.output_dir,
        filename="checkpoint-{epoch}",
        monitor="val_loss",
        verbose=True,
        mode="min",
        save_top_k=5
    )

    train_params = dict(
        accumulate_grad_batches=args.gradient_accumulation_steps,
        gpus=args.n_gpu,
        max_epochs=args.num_train_epochs,
        # early_stop_callback=False,
        # gradient_clip_val=args.max_grad_norm,
        # checkpoint_callback=checkpoint_callback,
        callbacks=[LoggingCallback(), checkpoint_callback],
        # logger=TensorBoardLogger(f'{args.output_dir}/logs'),
        num_sanity_val_steps=args.nb_sanity_val_steps,  # skip sanity check to save time for debugging purpose
        # plugins='ddp_sharded',
        progress_bar_refresh_rate=1,

    )

    print("Initialize model")
    model = T5FineTuner(args)
    #model = T5FineTuner(**train_args)

    trainer = pl.Trainer(**train_params)
    print(" Training model")
    trainer.fit(model)

    print("training finished")
    # print("Saving model")
    # model.model.save_pretrained(args.output_dir)

    # print("Saved model")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def parse_arguments():
    p = ArgumentParser()
                  
    p.add_argument('-t', '--trials', type=int, default=1,
                  help='number of trials for hyperparameter search')
    p.add_argument('--seed', type=int, default=42, help='randomization seed')
    p.add_argument('--features_kwargs', default= {
    # 'WordRatioFeature': {'target_ratio': 0.8},
    'CharRatioFeature': {'target_ratio': 0.8},
    'LevenshteinRatioFeature': {'target_ratio': 0.8},
    'WordRankRatioFeature': {'target_ratio': 0.8},
    'DependencyTreeDepthRatioFeature': {'target_ratio': 0.8}
})
    p = T5FineTuner.add_model_specific_args(p)
    p = pl.Trainer.add_argparse_args(p)
    args,_ = p.parse_known_args()
    return args

In [ ]:
from pytorch_lightning import callbacks
# -- fix path --
from pathlib import Path
import sys
# -- end fix path --
import torch
#from preprocessor import TURKCORPUS_DATASET, EXP_DIR, Preprocessor
import time
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import json
from contextlib import contextmanager
#from Ts_T5 import train
class MetricsCallback(pl.Callback):
  def __init__(self):
    super().__init__()
    self.metrics = []
  
  def on_validation_end(self, trainer, pl_module):
      self.metrics.append(trainer.callback_metrics)


def get_experiment_dir(create_dir=False):
    dir_name = f'{int(time.time() * 1000000)}'
    path = EXP_DIR / f'exp_{dir_name}'
    if create_dir == True: path.mkdir(parents=True, exist_ok=True)
    return path

def log_params(filepath, kwargs):
    filepath = Path(filepath)
    kwargs_str = dict()
    for key in kwargs:
        kwargs_str[key] = str(kwargs[key])
    json.dump(kwargs_str, filepath.open('w'), indent=4)

@contextmanager
def log_stdout(filepath, mute_stdout=False):
    '''Context manager to write both to stdout and to a file'''

    class MultipleStreamsWriter:
        def __init__(self, streams):
            self.streams = streams

        def write(self, message):
            for stream in self.streams:
                stream.write(message)

        def flush(self):
            for stream in self.streams:
                stream.flush()

    save_stdout = sys.stdout
    log_file = open(filepath, 'w')
    if mute_stdout:
        sys.stdout = MultipleStreamsWriter([log_file])  # Write to file only
    else:
        sys.stdout = MultipleStreamsWriter([save_stdout, log_file])  # Write to both stdout and file
    try:
        yield
    finally:
        sys.stdout = save_stdout
        log_file.close()


# def run_training(args_dict, dataset=WIKI_DATASET):

#     args_dict['output_dir'] = get_experiment_dir(create_dir=True)
#     log_params(args_dict["output_dir"] / "params.json", args_dict)

#     preprocessor = Preprocessor(args_dict['features_kwargs'])
#     preprocessor.preprocess_dataset(dataset)
#     args_dict["dataset"] = dataset
#     with log_stdout(args_dict['output_dir'] / "logs.txt"):
#         train(args_dict)

def run_training(args, dataset=WIKI_DATASET):

    args.output_dir = get_experiment_dir(create_dir=True)
    log_params(args.output_dir / "params.json", vars(args))

    preprocessor = Preprocessor(args.features_kwargs)
    preprocessor.preprocess_dataset(dataset)
    args.dataset = dataset
    with log_stdout(args.output_dir / "logs.txt"):
        train(args)

dataset = WIKI_DATASET
# args_dict = dict(
#     model_name='t5-base',
#     max_seq_length=256,
#     learning_rate=3e-4,
#     weight_decay=0.1,
#     adam_epsilon=1e-8,
#     warmup_steps=4,
#     train_batch_size=6,
#     valid_batch_size=6,
#     num_train_epochs=2,
#     custom_loss=False,   
#     gradient_accumulation_steps=1, #16
#     n_gpu=torch.cuda.device_count(),
#     # early_stop_callback=False,
#     seed=42,
#     nb_sanity_val_steps=0,
#     train_sample_size=1, # 1 = 100%, 0.5 = 50%
#     valid_sample_size=1, 
# )

# features_kwargs = {
#     # 'WordRatioFeature': {'target_ratio': 0.8},
#     'CharRatioFeature': {'target_ratio': 0.8},
#     'LevenshteinRatioFeature': {'target_ratio': 0.8},
#     'WordRankRatioFeature': {'target_ratio': 0.8},
#     'DependencyTreeDepthRatioFeature': {'target_ratio': 0.8}
# }
# args_dict['features_kwargs'] = features_kwargs
args = parse_arguments()
# run_training(args, dataset)
##########################

args.output_dir = get_experiment_dir(create_dir=True)
log_params(args.output_dir / "params.json", vars(args))

preprocessor = Preprocessor(args.features_kwargs)
preprocessor.preprocess_dataset(dataset)
args.dataset = dataset


def optuna_obj(trial, args):
  #args.learning_rate = trial.suggest_categorical('lr',[1e-6,1e-5])
  args.weight_decay = trial.suggest_categorical('weight_decay',[0.01,0.05])
  metrics_callback = MetricsCallback()
  cp_callback = pl.callbacks.ModelCheckpoint(
      dirpath=args.output_dir,
        filename="checkpoint-{epoch}",
        monitor="val_loss",
        verbose=True,
        mode="min",
        save_top_k=1
  )
  pr_callback = PyTorchLightningPruningCallback(trial,monitor = 'val_loss')
  model = T5FineTuner(args)

  # train_params = dict(
  #       accumulate_grad_batches=args.gradient_accumulation_steps,
  #       gpus=args.n_gpu,
  #       max_epochs=args.num_train_epochs,
  #       # early_stop_callback=False,
  #       # gradient_clip_val=args.max_grad_norm,
  #       # checkpoint_callback=checkpoint_callback,
  #       callbacks=[pr_callback, cp_callback,
  #                  metrics_callback],
  #       # logger=TensorBoardLogger(f'{args.output_dir}/logs'),
  #       num_sanity_val_steps=args.nb_sanity_val_steps,  # skip sanity check to save time for debugging purpose
  #       # plugins='ddp_sharded',
  #       #progress_bar_refresh_rate=1,

  # )
  early_stopping = EarlyStopping(
      monitor='val_loss',
      patience=10
  )
  trainer = pl.Trainer.from_argparse_args(
      args,
      callbacks=[pr_callback, cp_callback,
                 metrics_callback, early_stopping],
      num_sanity_val_steps=args.nb_sanity_val_steps
  )
  print(1)
  trainer.fit(model)
  print(2)
  #trainer.test(model)
  return min([x['val_loss'].item() for x in metrics_callback.metrics])

pruner = optuna.pruners.MedianPruner()
study = optuna.create_study(direction='maximize',pruner=pruner)
study.optimize(lambda x: optuna_obj(x, args),n_trials=args.trials)


[I 2022-03-04 08:13:08,881] A new study created in memory with name: no-name-c440cedb-7653-46e9-9c07-b4ab911d84ae


Preprocessing dataset: wiki
Preprocessing dataset "wiki" is finished.


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


1


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /content/text_sum_sim/experiments/exp_1646381588873793 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


...

Epoch 2, global step 2: val_loss was not in top 1


Sari score:  29.23294235595311
Val_loss 0.7076705764404689


Validating: 0it [00:00, ?it/s]

Epoch 3, global step 3: val_loss was not in top 1
[I 2022-03-04 08:21:32,426] Trial 0 finished with value: 0.691087007522583 and parameters: {'weight_decay': 0.01}. Best is trial 0 with value: 0.691087007522583.


Sari score:  29.23294235595311
Val_loss 0.7076705764404689
2


In [ ]:
study.best_trial.number

0

In [ ]:
cp_filepath = 'checkpoint-epoch='+str(study.best_trial.number)+'.ckpt'
bestcp_file = args.output_dir / cp_filepath
bestcp_file

PosixPath('/content/text_sum_sim/experiments/exp_1646381588873793/checkpoint-epoch=0.ckpt')

In [ ]:
model = T5FineTuner.load_from_checkpoint(str(bestcp_file))
model.save_core_model()
complex_sent = '''Jeddah is the principal gateway to Mecca , Islam 's holiest city , which able-bodied Muslims are required to visit at least once in their lifetime .'''
pred = model.generate(complex_sent)
pred

"Jeddah is the principal gateway to Mecca, Islam's holiest city. able-bodied Muslims are required to visit at least once in their lifetime."